In [ ]:
import os
import pandas as pd
import math
import matplotlib.pyplot as plt
import numpy as np
import random

from functionsFIMS import getAnalysisNumbers
from runDataClass import runData
from polyaClass import myPolya

In [ ]:
if __name__ == '__main__':
    runNos = getAnalysisNumbers()
    simData = runData(runNos[0])
    treenames = simData.getTreeNames()
    print(treenames)
    for name in treenames:
        simData.printColumns(name)

In [ ]:
if __name__ == '__main__':
    binWidth=3
    simData.plotAvalancheSize(binWidth)
    simData.plotAvalancheFits(binWidth)

    thresh = 10
    fitPolya = myPolya(24.701, 0.488)
    efficiency = fitPolya.calcEfficiencyErrs(thresh, 0.671, 0.095)
    print(f'{efficiency[0]:.3f} (+{efficiency[1]:.3f}, {efficiency[2]:.3f})')

In [ ]:
if __name__ == '__main__':
    simData.plotAvalanche2D(random.randint(0, 999), plotName='Random')

In [ ]:
TESTDATA = runData(1143)

In [ ]:
def plotAvalancheSignal(runData, avalancheID=0):
    """
    """

    allData = runData.getDataFrame('signalData')
    singleData = allData[allData['Avalanche ID']==avalancheID]
    
    gain = runData._getAvalancheGain(avalancheID)
    
    if singleData is None:
        print(f"An error occured plotting ID='{avalancheID}'.")
        return

    totalCharge = singleData['Signal Strength'].sum()

    # Create figure
    fig = plt.figure()
    fig.suptitle(f'Induced Signal from Avalanche: {avalancheID} (Gain={gain}, Total Charge={totalCharge:.3f})')
    ax = fig.add_subplot(111)
    
    ax.plot(
        singleData['Signal Time'], singleData['Signal Strength']
    )

    ax.set_xlabel('Time (ns)')
    ax.set_ylabel('Signal Strength (C/ns)')#TODO - The units here seem weird

    plt.grid()

    plt.tight_layout()   
    
    return fig

In [ ]:
_ = plotAvalancheSignal(TESTDATA, avalancheID=10)